In [ ]:
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
from pathlib import Path

path = str(Path().resolve())
path = path + "\\ADNI_PROCESSED"

def apply_mask(img_n_mmni, img_mask):
    """
        Taking a n_mmni and apply the correspondant mask
        param:
            img_n_mmi   : image n_mmi
            img_mask    : mask
    """
    mmni_m = img_n_mmni.get_fdata()
    mask_m = img_mask.get_fdata().astype(bool)
    mask_bg = np.logical_not(mask_m)
    mmni_m[mask_bg] = 0
    return mmni_m

def process_irm_data():
    """
        Create a new directory and process all images from tha ADNI1 directory
    """
    path = str(Path().resolve())
    path_res = path + "\\ADNI_PROCESSED"
    Path(path_res).mkdir(parents=True, exist_ok=True) # Create a directory for data processed
    path = path + "\\ADNI1"
    for filename in os.listdir(path):
        if filename.startswith("n_mmni"):
            n_mmni_filename = os.path.join(path, filename)
            mask_filename = os.path.join(path, "mask_" + filename)
            img_n_mmni = nib.load(n_mmni_filename)
            img_mask = nib.load(mask_filename)
            n_mmni_mask = apply_mask(img_n_mmni, img_mask)
            img = nib.Nifti1Image(n_mmni_mask, np.eye(4))
            nib.save(img, os.path.join(path_res, filename))

# process_irm_data()

In [ ]:
def load_processed_data(path):
    """
        load all n_mmni found in the path
    """
    if not os.path.isdir(path):
        print("Can't found directory: " + path)
    else:
        list_x = []
        for filename in os.listdir(path):
            n_mmni_filename = os.path.join(path, filename)
            img_n_mmni = nib.load(n_mmni_filename)
            mmni_matrix = img_n_mmni.get_fdata()
            list_x.append((filename, mmni_matrix))
        return list_x

# Not tested yet; crashed last time
# path = str(Path().resolve())
# path_to_data_proc = path + "\\ADNI_PROCESSED"
# X = load_processed_data(path_to_data_proc)

In [ ]:
def cut_2D_i(img_n_mmni, axe, idx):
    """
        Function that returns a 2D cut from the "img" in the index "idx", along the axe given in parameter
    """
    axe_dim = {"x": img_n_mmni.shape[0], "y": img_n_mmni.shape[1], "z":img_n_mmni.shape[2]}
    if axe_dim[axe] <= idx or idx < 0:
        print("Invalid value for index must be between 0 and " , axe_dim[axe])
        return
    if axe == "x":
        cropped_img = img_n_mmni.slicer[idx:idx+1, ...]
    elif axe == "y":
        cropped_img = img_n_mmni.slicer[:, idx:idx+1,:]
    elif axe == "z":
        cropped_img = img_n_mmni.slicer[..., idx:idx+1]
    else:
        print("Choose a valid value for axe: x, y or z")
    return cropped_img

def patch_3D(img_n_mmni, axe, idx_start, idx_end):
    """
        Function that returns a 3D patch from the "img" along the axe given in parameter, from the idx_start to idx_end
    """
    axe_dim = {"x": img_n_mmni.shape[0], "y": img_n_mmni.shape[1], "z":img_n_mmni.shape[2]}
    if axe_dim[axe] <= idx_start or idx_start < 0 or axe_dim[axe] <= idx_end or idx_end < 0 or idx_start >= idx_end:
        print("Invalid value for index must, values must be between 0 and " , axe_dim[axe], "and idx_start must be greater than idx_end")
        return
    if axe == "x":
        cropped_img = img_n_mmni.slicer[idx_start:idx_end, ...]
    elif axe == "y":
        cropped_img = img_n_mmni.slicer[:, idx_start:idx_end,:]
    elif axe == "z":
        cropped_img = img_n_mmni.slicer[..., idx_start:idx_end]
    else:
        print("Choose a valid value for axe: x, y or z")
    return cropped_img

# To test thid function
n_mmni_filename = os.path.join(path, "n_mmni_fADNI_002_S_0295_1.5T_t1w.nii.gz")
img_n_mmni = nib.load(n_mmni_filename)
crop_img = patch_3D(img_n_mmni, "z", 90, 120)
print(crop_img.shape)
IM_HEIGHT = crop_img.shape[0]
IM_WIDTH = crop_img.shape[1]
nib.save(crop_img, 'test_image.nii')

In [ ]:
def load_X_data(path):
    if not os.path.isdir(path):
        print("Can't found directory: " + path)
    else:
        list_x = []
        for filename in os.listdir(path):
            n_mmni_filename = os.path.join(path, filename)
            img_n_mmni = nib.load(n_mmni_filename)
            # Customize your choice: taking a 2D cuts or 3D patches
            cropped_img = cut_2D_i(img_n_mmni, "z", 90)
            cropped_n_mmni_matrix = cropped_img.get_fdata()
            list_x.append((filename, cropped_n_mmni_matrix))
        return list_x

X_data = load_X_data(path)

In [ ]:
import os
import re
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

def load_data(path):
    data = pd.read_csv(path, names= ['Subject ID', 'Rooster ID', 'Age', 'Sexe', 'Group', 'Conversion', 'MMSE', 'RAVLT', 'FAQ', 'CDR-SB', 'ADAS11'], usecols = ['Subject ID', 'Rooster ID', 'Group'])
    data.index = data['Subject ID']
    data = data.drop(['Subject ID'], axis=1) 
    return data

path = str(Path().resolve())
path = path + "\\ADNI1\\list_standardized_tongtong_2017.csv"
y_data = load_data(path)

def process_data(data):
    ordinal_encoder = OrdinalEncoder()
    data = data[(data.Group == 'CN') | (data.Group == 'AD')]
    data_cat = data[['Group']] 
    data_cat_encoded = ordinal_encoder.fit_transform(data_cat)
    data['Group'] = data_cat_encoded
    return data

y_data = process_data(y_data)
y_data.head(10)

In [ ]:
usecols = ['Subject ID', 'Rooster ID', 'Group']
def prepare_X_of_Y(Y_data):
    X_data = [] #np.zeros((len(Y_data), IM_HEIGHT, IM_WIDTH, 1), dtype=np.float32)
    path = str(Path().resolve())
    path += "\\ADNI_PROCESSED"
    for index, row in Y_data.iterrows():
        file = path + '\\n_mmni_fADNI_' + index + '_1.5T_t1w.nii.gz'
        if os.path.isfile(file):
            img_n_mmni = nib.load(file)
            crop_img = cut_2D_i(img_n_mmni, "z", 90)
            X_data.append(crop_img.get_fdata())
        else:
            print("droped")
            Y_data.drop(index, inplace=True)
    return X_data, Y_data

X_data, Y_data = prepare_X_of_Y(y_data)

# U-Net Neural Network

Creation of two U-Net models:
* One for 2D inputs, in case we slice the input into 2D images.
* The other for 3D inputs, in case we use 3D blocs of the image.

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Cropping2D, Conv3D, MaxPooling3D, UpSampling3D, Cropping3D, Input, concatenate
from tensorflow.keras.models import Model

def create_Unet_model2D(input_size, depth=5, padding='valid'):
    inputs = Input(shape=input_size)
    x = inputs
    num_filters = 64
    encode_layers_list = []
    for i in range(depth):
        x = Conv2D(filters=num_filters, kernel_size=(3,3), padding=padding, activation='relu')(x)
        x = Conv2D(filters=num_filters, kernel_size=(3,3), padding=padding, activation='relu')(x)
        if i != depth - 1:
            encode_layers_list.append(x)
            x = MaxPooling2D(pool_size=(2,2), strides=2)(x)
            num_filters *= 2
    
    for i in range(depth - 1):
        x = UpSampling2D()(x)
        x = Conv2D(filters=num_filters, kernel_size=(2,2), padding='same', activation='relu')(x)
        #cropping
        encoder_shape = encode_layers_list[depth - 2 - i].shape
        decoder_shape = x.shape
        shape_diff = (encoder_shape[1] - decoder_shape[1], encoder_shape[2] - decoder_shape[2])
        xshape_diff = shape_diff[0] // 2
        yshape_diff = shape_diff[1] // 2
        if shape_diff[0] % 2 != 0:
            xshape_diff = (shape_diff[0] // 2, shape_diff[0] // 2 + 1)
        if shape_diff[1] % 2 != 0:
            yshape_diff = (shape_diff[1] // 2, shape_diff[1] // 2 + 1)
        croped_layer = Cropping2D(cropping=(xshape_diff, yshape_diff))(encode_layers_list[depth - 2 - i])
        
        x = concatenate([croped_layer, x])
        num_filters /= 2
        x = Conv2D(filters=num_filters, kernel_size=(3,3), padding=padding, activation='relu')(x)
        x = Conv2D(filters=num_filters, kernel_size=(3,3), padding=padding, activation='relu')(x)

    outputs = Conv2D(filters=2, padding=padding, kernel_size=(1,1))(x)
        
    return Model(inputs, outputs)

def create_Unet_model3D(input_size, depth=5, padding='valid'):
    inputs = Input(shape=input_size)
    x = inputs
    num_filters = 64
    encode_layers_list = []
    for i in range(depth):
        x = Conv3D(filters=num_filters, kernel_size=(3,3,3), padding=padding, activation='relu')(x)
        x = Conv3D(filters=num_filters, kernel_size=(3,3,3), padding=padding, activation='relu')(x)
        if i != depth - 1:
            encode_layers_list.append(x)
            x = MaxPooling3D(pool_size=(2,2,2), strides=2)(x)
            num_filters *= 2
    

    for i in range(depth - 1):
        x = UpSampling3D()(x)
        x = Conv3D(filters=num_filters, kernel_size=(2,2,2), activation='relu', padding='same')(x)
        #cropping
        encoder_shape = encode_layers_list[depth - 2 - i].shape
        decoder_shape = x.shape
        shape_diff = (encoder_shape[1] - decoder_shape[1], encoder_shape[2] - decoder_shape[2], encoder_shape[3] - decoder_shape[3])
        xshape_diff = shape_diff[0] // 2
        yshape_diff = shape_diff[1] // 2
        zshape_diff = shape_diff[2] // 2
        if shape_diff[0] % 2 != 0:
            xshape_diff = (shape_diff[0] // 2, shape_diff[0] // 2 + 1)
        if shape_diff[1] % 2 != 0:
            yshape_diff = (shape_diff[1] // 2, shape_diff[1] // 2 + 1)
        if shape_diff[2] % 2 != 0:
            zshape_diff = (shape_diff[2] // 2, shape_diff[2] // 2 + 1)
        croped_layer = Cropping3D(cropping=(xshape_diff, yshape_diff, zshape_diff))(encode_layers_list[depth - 2 - i])

        x = concatenate([croped_layer, x])
        num_filters /= 2
        x = Conv3D(filters=num_filters, kernel_size=(3,3,3), padding=padding, activation='relu')(x)
        x = Conv3D(filters=num_filters, kernel_size=(3,3,3), padding=padding, activation='relu')(x)

    outputs = Conv3D(filters=2, kernel_size=(1,1,1))(x)
        
    return Model(inputs, outputs)

In [ ]:
model_2d = create_Unet_model2D((572,572,1))
model_2d.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2d.summary()